# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import base64
import tiktoken
import base64
import io
import json
import os
from pdf2image import convert_from_path
import PyPDF2
import requests
import shutil
from fastcore.all import *

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def split_pdf(file_path, every=1, until=None, output_folder=None):
    """
    Split a PDF file into multiple smaller PDF files.

    Args:
        file_path (str): The path to the input PDF file.
        every (int, optional): The interval at which to split the PDF. Default is 1.
        until (int, optional): The page number until which to split the PDF. Default is None.
        output_folder (str, optional): The folder where the split PDF files will be saved. Default is None.

    Returns:
        int: The number of splits performed.

    """
    pdf_file = PyPDF2.PdfFileReader(file_path)
    pdf_writer = PyPDF2.PdfFileWriter()

    num_splits = 0
    for page in range(pdf_file.getNumPages()):
        pdf_writer.addPage(pdf_file.getPage(page))

        # If it's the n page or the 'until' page, save the current selection
        if until is None:
            until = pdf_file.getNumPages()
        if output_folder is None:
            output_folder = os.path.dirname(file_path)
        basename = os.path.splitext(os.path.basename(file_path))[0]
        if page % every == 0 or page == until:
            output_file = f"{output_folder}/{basename}_page_{page}.pdf"
            with open(output_file, "wb") as out:
                pdf_writer.write(out)
            pdf_writer = PyPDF2.PdfFileWriter()
            num_splits += 1
    return num_splits

In [ ]:
# test
path = '../example_data/1.pdf'
foo = split_pdf(path, 1)
test_eq(foo, 2)
# remove temporary files
for f in os.listdir(os.path.dirname(path)):
    if f.startswith('1_page_'):
        os.remove(f'{os.path.dirname(path)}/{f}')

In [ ]:
#| export
def encode_image(image_path):
  """Function to encode the image"""
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Test
image_path = '../example_data/1.jpg'
image_base64 = encode_image(image_path)
test_eq_type(type(image_base64), str)

In [ ]:
#| export
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

In [ ]:
# test
image_path = '../example_data/1.jpg'
base64_image = encode_image(image_path)
messages = [{"role": "user", "content": "Hello, how are you?", "image": base64_image}]
num_tokens = num_tokens_from_messages(messages)
num_tokens


351620

In [ ]:
#| export
DEFAULT_SYSTEM_PROMPT = ("You are a helpful assistant designed to see an exam and output JSON \
with the extracted information. You will be given an image of the exam.")

DEFAULT_USER_PROMPT = ("Extract the last name (Apellidos in Spanish) and the first name \
(Nombre in Spanish) from the top of the image, and turn them into upper case. \
You will find them handwritten after the labels `Apellidos` and `Nombre` respectively. \
The fields of your JSON output will have those exact same label names")

In [ ]:
#| export
def extract_student_info(path_pdf: str, crop=None, api_key = None, 
                 model="gpt-4o", prompt=DEFAULT_USER_PROMPT, 
                 system_prompt=DEFAULT_SYSTEM_PROMPT, verbose=False):
    """
    Extracts student information from a PDF file using OpenAI's chatGPT.

    Args:
        path_pdf (str): The path to the PDF file.
        crop (tuple, optional): Tuple with the coordinates of the crop (left, top, right, bottom).
        api_key (str, optional): The API key for accessing the OpenAI API. If 
            not provided, the function will try to use the OPENAI_API_KEY environment variable.
        model (str, optional): The model to use for generating responses.
        prompt (str, optional): The user prompt to start the conversation.
        system_prompt (str, optional): The system prompt to provide context to the model.
        verbose (bool, optional): Whether to print information about the process.

    Returns:
        str: The extracted information from the PDF.

    Raises:
        KeyError: If the OPENAI_API_KEY environment variable is not set.
        IndexError: If the response from the OpenAI API does not contain any choices.

    """
    with open(path_pdf, 'rb') as file:
        pdf = PyPDF2.PdfFileReader(file)

        # Check if the PDF has more than one page
        if pdf.getNumPages() > 1 and verbose:
            print("The PDF has more than one page. Only the first page will be \
                  converted to an image.")
        
        # Convert the PDF to images and get the first one
        image = convert_from_path(path_pdf, last_page=1, first_page=0, fmt='jpeg')[0]

        # Crop the image
        if crop:
            image = image.crop(crop)
        
        # Convert to base64
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ifnone(api_key, os.environ.get('OPENAI_API_KEY'))}"
        }
        payload = {
            "model": f'{model}',
            "response_format": { "type": "json_object" },
            "messages": [
                {
                    "role": "system", 
                    "content": f"{system_prompt}",   
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"{prompt}"
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", 
                                 headers=headers, json=payload)
        
        return json.loads(response.json()['choices'][0]['message']['content'])

In [ ]:
# test
path_pdf = '../example_data/1.pdf'
info = extract_student_info(path_pdf)
info

{'Apellidos': 'RODRIGUEZ FERNANDEZ', 'Nombre': 'VICTOR'}

In [ ]:
#| export
@delegates(extract_student_info)
def rename_exam_file(pdf_path: str, output_path: str, keep_old=True, **kwargs):
    """
    Extracts student information from an exam PDF file using OpenAI's chatGPT and renames that
    file with the extracted information as the name.

    Parameters:
    - pdf_path (str): The path to the input PDF file.
    - output_path (str): The path to the directory where the renamed file will be saved.
    - **kwargs: Additional keyword arguments to be passed to the `extract_student_info` function.
    - keep_old (bool): Whether to keep the old file or not.

    Returns:
    - new_name (str): The new name of the renamed file.

    Example:
    >>> rename_exam_file('/path/to/input.pdf', '/path/to/output', option1='value1', option2='value2')
    'Doe_John.pdf'
    """
    info = extract_student_info(pdf_path, **kwargs)
    new_name = f"{info['Apellidos']}_{info['Nombre']}.pdf"
    new_file_path = os.path.join(output_path, new_name)
    if not keep_old:
        os.rename(pdf_path, new_file_path)
    else:
        os.makedirs(output_path, exist_ok=True)
        shutil.copy(pdf_path, new_file_path)
        
    return new_name

In [ ]:
# Test renaming without keeping old file
path_pdf = '../example_data/1.pdf'
output_path = '../example_data'
new_name = rename_exam_file(pdf_path=path_pdf, output_path=output_path, keep_old=False)
new_name

'RODRIGUEZ FERNANDEZ_VICTOR.pdf'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()